<a href="https://colab.research.google.com/github/Oleg007003/A-model-for-predicting-the-outcome-of-a-football-game/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок с загрузкой данных

In [172]:
!pip install catboost

In [173]:
import numpy as np
from matplotlib import pyplot as plt
import catboost as cb
import keras
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import normalize
import math
from pandas.plotting import table 

In [174]:
data = pd.read_csv('http://assets.codeforces.com/rounds/1522/3056cab52876d3327edafe4771c8589a96cb1a93/train.csv')
del data[data.columns[0]]

In [175]:
data.head(15)

,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef
0,0,NaN,152.0,426.0,0.0,0.0,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.627690,3.577721,2.198600
1,0,NaN,216.0,341.0,2.0,2.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.608321,4.251999,6.533805
2,0,NaN,223.0,119.0,2.0,2.0,2.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166811,3.486596,3.821467
3,0,NaN,68.0,137.0,2.0,1.0,0.0,1.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.259244,3.592401,3.460103
4,1,NaN,425.0,225.0,2.0,2.0,1.0,0.0,-1,13.0,11.0,5.0,7.0,18.0,15.0,4.0,10.0,3.0,4.0,1.0,0.0,2.424278,3.414191,3.282876
5,1,NaN,393.0,300.0,0.0,0.0,0.0,0.0,-1,6.0,6.0,1.0,2.0,9.0,13.0,6.0,8.0,1.0,0.0,0.0,0.0,2.787163,3.375001,2.817568
6,1,NaN,239.0,140.0,0.0,0.0,0.0,0.0,-1,16.0,4.0,8.0,1.0,9.0,16.0,6.0,1.0,0.0,2.0,0.0,0.0,2.080239,3.582070,3.998117
7,1,NaN,211.0,424.0,2.0,0.0,1.0,0.0,-1,18.0,5.0,7.0,3.0,12.0,10.0,6.0,6.0,2.0,1.0,0.0,1.0,2.099892,3.449822,4.099789
8,1,NaN,99.0,5.0,1.0,0.0,0.0,0.0,-1,9.0,5.0,5.0,2.0,5.0,9.0,6.0,4.0,0.0,1.0,0.0,0.0,1.312957,5.807312,13.129574
9,0,NaN,303.0,382.0,1.0,0.0,1.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.528475,4.476952,7.553624


In [176]:
for i in range(9, 21):
  name = data.columns[i]
  data[name] = data[name].fillna(data[name].mean())

In [177]:
data['is_draw'] = (data['full_time_home_goals'] == data['full_time_away_goals']) + 0
data['is_win'] = (data['full_time_home_goals'] > data['full_time_away_goals']) + 0
data['is_lose'] = (data['full_time_home_goals'] < data['full_time_away_goals']) + 0
data['result'] = data['is_win'] - data['is_lose']

In [178]:
data = data[(data['Division'] >= 0) & (data['Division'] <= 9)]
data, data_test = train_test_split(data, test_size = 0.15, random_state= 15, shuffle = True)

In [179]:
y = data['result'].copy()
X = data.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1).copy()
Priznak = data.copy()
del Priznak['result']

In [180]:
comands_home = data['home_team'].unique()
comands_away = data['away_team'].unique()

In [181]:
tX = Priznak.copy()
tX['ans'] = y
cr = tX.corr()
cr  

,Division,Time,home_team,away_team,full_time_home_goals,full_time_away_goals,half_time_home_goals,half_time_away_goals,Referee,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_corners,away_corners,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_coef,draw_coef,away_coef,is_draw,is_win,is_lose,ans
Division,1.000000,NaN,0.028651,0.031453,-0.030846,-0.043565,-0.014284,-0.027326,0.255949,-0.005208,-0.008211,0.006959,0.004912,-0.111590,-0.101982,0.020798,0.026309,-0.086121,-0.066190,-0.040432,-0.018025,-0.024717,-0.135642,-0.030124,0.005222,0.003539,-0.008892,0.006880
Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_team,0.028651,NaN,1.000000,-0.013134,-0.009561,-0.007813,-0.012457,-0.010626,-0.000700,-0.026123,-0.004062,-0.023679,0.004468,0.020173,0.011741,-0.010435,0.005353,0.011538,0.012806,0.011963,0.025443,-0.010251,-0.028000,-0.006728,-0.002262,0.002807,-0.000865,0.002114
away_team,0.031453,NaN,-0.013134,1.000000,-0.002630,-0.010121,-0.002009,-0.007653,0.004137,-0.016448,-0.020762,-0.007805,-0.005970,-0.024342,0.031643,-0.003997,-0.001239,-0.002065,0.021775,-0.004195,0.017566,-0.029763,-0.025461,-0.001805,0.006583,0.011703,-0.019079,0.017172
full_time_home_goals,-0.030846,NaN,-0.009561,-0.002630,1.000000,-0.043032,0.677235,-0.020662,-0.017446,0.174068,-0.045285,0.347726,-0.048409,-0.002984,-0.034916,-0.012671,-0.018715,-0.043204,0.015376,-0.051470,0.069928,-0.197956,0.181161,0.254073,-0.228786,0.635274,-0.469367,0.626345
full_time_away_goals,-0.043565,NaN,-0.007813,-0.010121,-0.043032,1.000000,-0.030313,0.677639,0.001125,-0.052787,0.185416,-0.046624,0.357244,-0.019915,-0.015636,-0.012371,0.017403,0.048865,-0.005870,0.085679,-0.056280,0.235184,-0.039666,-0.195449,-0.088753,-0.489000,0.617212,-0.620336
half_time_home_goals,-0.014284,NaN,-0.012457,-0.002009,0.677235,-0.030313,1.000000,-0.037668,-0.002514,0.070064,0.000774,0.208341,-0.023132,0.020984,-0.032065,-0.052278,0.032231,-0.034033,0.003556,-0.035170,0.025195,-0.138717,0.121331,0.172229,-0.147721,0.420453,-0.314224,0.416475
half_time_away_goals,-0.027326,NaN,-0.010626,-0.007653,-0.020662,0.677639,-0.037668,1.000000,0.001846,-0.000624,0.086237,-0.011840,0.220960,-0.015367,0.010609,0.034184,-0.025900,0.038931,0.007226,0.035219,-0.026848,0.169254,-0.026417,-0.136355,-0.063396,-0.323063,0.412368,-0.412316
Referee,0.255949,NaN,-0.000700,0.004137,-0.017446,0.001125,-0.002514,0.001846,1.000000,-0.029980,0.004840,-0.072267,-0.036655,-0.166123,-0.158821,0.011335,0.039331,-0.122072,-0.100828,-0.028728,-0.034271,0.000468,-0.027921,-0.055996,-0.007037,-0.009348,0.016959,-0.014645
home_shots,-0.005208,NaN,-0.026123,-0.016448,0.174068,-0.052787,0.070064,-0.000624,-0.029980,1.000000,-0.118713,0.627566,-0.145358,-0.083505,0.018122,0.386512,-0.226620,-0.101770,0.044978,-0.104745,0.102789,-0.175967,0.104379,0.184881,-0.003951,0.103779,-0.108972,0.119762


# Блок с полученением подрандомного датасета

In [182]:
def getResults(podPriz):
  tresults_games_shots = np.zeros((495,495))
  tresults_games_itog = np.zeros((495,495))
  tgames = podPriz[['home_team', 'away_team', 'full_time_home_goals', 'full_time_away_goals', 'is_win', 'is_lose']]
  for index, row in tgames.iterrows():
    hteam = int(row['home_team'])
    ateam = int(row['away_team'])
    hshots = int(row['full_time_home_goals'])
    ashots = int(row['full_time_away_goals'])
    hfinal = int(row['is_win'] - row['is_lose'])
    tresults_games_shots[hteam][ateam] += hshots - ashots
    tresults_games_shots[ateam][hteam] += ashots - hshots
    tresults_games_itog[hteam][ateam] += hfinal
    tresults_games_itog[ateam][hteam] -= hfinal
  return tresults_games_shots, tresults_games_itog

In [183]:
name_home_priznak = ['home_team', 'Division', 'full_time_home_goals', 'half_time_home_goals', 'home_shots', 'home_shots_on_target', 'home_fouls', 'home_corners', 'home_yellow_cards', 'home_red_cards', 'home_coef']
name_away_priznak = ['away_team', 'full_time_away_goals', 'half_time_away_goals', 'away_shots', 'away_shots_on_target', 'away_fouls', 'away_corners', 'away_yellow_cards', 'away_red_cards', 'away_coef']
name_draw_priznak = ['draw_coef']
names_home = ['players_full_time_home_goals', 'players_half_time_home_goals',
       'players_home_shots', 'players_home_shots_on_target',
       'players_home_fouls', 'players_home_corners',
       'players_home_yellow_cards', 'players_home_red_cards',
       'players_home_coef']
names_away = ['players_full_time_away_goals',
       'players_half_time_away_goals', 'players_away_shots',
       'players_away_shots_on_target', 'players_away_fouls',
       'players_away_corners', 'players_away_yellow_cards',
       'players_away_red_cards', 'players_away_coef']

In [184]:
def getPlayers(podPriznak):
  home_priznak = podPriznak[name_home_priznak]
  away_priznak = podPriznak[name_away_priznak]
  draw_priznak = podPriznak[name_draw_priznak]
  minPlayerId = min(min(data['home_team']), min(data['away_team'])) # == 1
  maxPlayerId = max(max(data['home_team']), max(data['away_team'])) # == 494

  players = pd.DataFrame()
  players['id'] = range(0, 495 + 1)
  lst_home = []
  lst_away = []
  for i in range(0, 495 + 1):
    lst_home.append(podPriznak[podPriznak['home_team'] == i].shape[0])
    lst_away.append(podPriznak[podPriznak['away_team'] == i].shape[0])
  players['games_home'] = pd.DataFrame(lst_home)
  players['games_away'] = pd.DataFrame(lst_away)
  p_home = pd.DataFrame()
  for cols in range(1, len(name_home_priznak)):
    name = name_home_priznak[cols]
    lst = [0]
    for i in range(1, 495 + 1):
      lst.append(home_priznak[home_priznak['home_team'] == i][name].sum())
    p_home['players_'+name] = lst

  p_away = pd.DataFrame()
  for cols in range(1, len(name_away_priznak)):
    name = name_away_priznak[cols]
    lst = [0]
    for i in range(1, 495 + 1):
      lst.append(away_priznak[away_priznak['away_team'] == i][name].sum())
    p_away['players_'+name] = lst
  
  players = players.drop(0)
  p_home = p_home.drop(0)
  p_away = p_away.drop(0)

  p_home[p_home.columns[0]] //= players['games_home']
  for i in range(1, len(p_home.columns)):
    p_home[p_home.columns[i]] /= players['games_home']

  for i in range(0, len(p_away.columns)):
    p_away[p_away.columns[i]] /= players['games_away']

  p_away.reset_index(drop=True, inplace=True)
  p_home.reset_index(drop=True, inplace=True)
  players.reset_index(drop=True, inplace=True)
  a = pd.concat([p_away, p_home], axis = 1)
  a.reset_index(drop=True, inplace=True)
  players = pd.concat([players, a], axis = 1)

  return players

In [216]:
X_train = pd.DataFrame()
y_train = pd.DataFrame()
for i in range(1, 20):
  print(i)
  data_train, data_valid = train_test_split(data, test_size = 0.95, random_state= i * 10 + 1, shuffle = True)
  yt = data_train['result'].copy()
  Xt = data_train.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1).copy()
  trainPriznak = data_valid.copy()
  trainPriznak = trainPriznak.sample(frac=0.15)
  del trainPriznak['result']
  results_games_shots, results_games_itog = getResults(trainPriznak)
  players = getPlayers(trainPriznak)
  df_h = pd.DataFrame()
  df_a = pd.DataFrame()
  df_h[names_home] = np.nan
  df_a[names_away] = np.nan
  for index, row in Xt.iterrows():
    home = (players[players['id'] == int(row['home_team'])][names_home])
    away = (players[players['id'] == int(row['away_team'])][names_away])
    df_h = df_h.append(home, ignore_index=True)
    df_a = df_a.append(away, ignore_index=True)
  X_r = Xt[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef']]
  X_r.reset_index(drop=True, inplace=True)
  a = pd.concat([df_h, df_a], axis = 1)
  a.reset_index(drop=True, inplace=True)
  X_r = pd.concat([X_r, a], axis = 1)
  teams_res_shots = np.zeros(X_r.shape[0])
  teams_res_itog = np.zeros(X_r.shape[0])
  for index, row in X_r.iterrows():
    hteam = int(row['home_team'])
    ateam = int(row['away_team'])
    teams_res_shots[index] = results_games_shots[hteam][ateam]
    teams_res_itog[index] = results_games_itog[hteam][ateam]
  X_r['personal_res_shots'] = teams_res_shots
  X_r['personal_res_itog'] = teams_res_itog
  X_r['strong_personal_res_shots'] = (teams_res_shots > 8) * 1
  X_r['weak_personal_shots'] = (teams_res_shots < -9) * 1
  X_r['personal_res_shots_in_qube'] = teams_res_shots ** 3
  X_r['personal_res_itog_in_qube'] = teams_res_itog ** 3
  X_train = pd.concat([X_train, X_r], axis = 0)
  y_train = pd.concat([y_train, yt], axis = 0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [217]:
X_train[X_train['home_team'] == 152]

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots,personal_res_shots_in_qube,personal_res_itog_in_qube
226,0,NaN,152.0,137.0,-1,2.865081,3.355951,2.754885,1.307692,0.769231,13.049802,4.828113,11.668640,5.768417,1.546420,0.078808,2.805045,2.250000,1.250000,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,3.833650,0.0,0.0,0,0,0.0,0.0
360,0,NaN,152.0,303.0,-1,3.264511,3.416585,2.433176,1.307692,0.769231,13.049802,4.828113,11.668640,5.768417,1.546420,0.078808,2.805045,1.333333,0.444444,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,3.488185,1.0,1.0,0,0,1.0,1.0
558,0,NaN,152.0,341.0,-1,4.521001,3.898107,1.878727,1.307692,0.769231,13.049802,4.828113,11.668640,5.768417,1.546420,0.078808,2.805045,1.800000,0.500000,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,4.064588,-1.0,-1.0,0,0,-1.0,-1.0
675,0,NaN,152.0,213.0,-1,3.036315,3.410794,2.580868,1.307692,0.769231,13.049802,4.828113,11.668640,5.768417,1.546420,0.078808,2.805045,1.500000,0.666667,11.349346,3.333552,13.119785,5.105664,1.985354,0.076782,3.676894,0.0,0.0,0,0,0.0,0.0
8,0,NaN,152.0,435.0,-1,4.290738,3.874851,1.927282,1.090909,0.545455,13.089598,5.006737,11.569931,5.612915,1.552508,0.069853,2.913696,0.500000,0.000000,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,2.224318,0.0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,0,NaN,152.0,177.0,-1,2.467603,3.410509,3.209891,1.272727,0.727273,13.089598,4.734009,12.569931,5.794734,1.643417,0.069853,2.857710,1.400000,0.400000,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,4.766145,-3.0,-2.0,0,0,-27.0,-8.0
90,0,NaN,152.0,4.0,-1,2.518661,3.351444,3.178794,1.600000,1.000000,14.109829,4.717698,13.112821,6.399295,2.340230,0.068301,3.163848,1.142857,0.000000,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,4.339714,-2.0,-1.0,0,0,-8.0,-1.0
432,0,NaN,152.0,426.0,-1,2.674589,3.330621,2.977373,1.600000,1.000000,14.109829,4.717698,13.112821,6.399295,2.340230,0.068301,3.163848,1.500000,0.500000,10.768014,4.000246,12.009758,4.368872,2.108523,0.336380,3.258586,0.0,0.0,0,0,0.0,0.0
117,0,NaN,152.0,177.0,-1,2.467603,3.410509,3.209891,1.000000,0.285714,12.062347,4.640802,13.815019,5.535085,1.339491,0.060983,3.449194,2.000000,1.333333,10.524018,4.000328,12.679677,4.658497,1.978031,0.115173,5.468612,0.0,0.0,0,0,0.0,0.0


In [218]:
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_train['Division'] = X_train['Division'].apply(str)
X_train['home_team'] = X_train['home_team'].apply(str)
X_train['away_team'] = X_train['away_team'].apply(str)
X_train['Referee'] = X_train['Referee'].apply(str)
print(X_train.dtypes)


Division                         object
Time                            float64
home_team                        object
away_team                        object
Referee                          object
home_coef                       float64
draw_coef                       float64
away_coef                       float64
players_full_time_home_goals    float64
players_half_time_home_goals    float64
players_home_shots              float64
players_home_shots_on_target    float64
players_home_fouls              float64
players_home_corners            float64
players_home_yellow_cards       float64
players_home_red_cards          float64
players_home_coef               float64
players_full_time_away_goals    float64
players_half_time_away_goals    float64
players_away_shots              float64
players_away_shots_on_target    float64
players_away_fouls              float64
players_away_corners            float64
players_away_yellow_cards       float64
players_away_red_cards          float64


In [219]:
y_test = data_test['result'].copy()
X_test = data_test.drop(['is_win', 'is_draw', 'is_lose', 'result', 'full_time_home_goals', 'full_time_away_goals', 'half_time_home_goals', 'half_time_away_goals'], axis = 1).copy()
testPriznak = data.copy()
del testPriznak['result']
results_games_shots_test, results_games_itog_test = getResults(podPriznak)
players_test = getPlayers(testPriznak)
df_h = pd.DataFrame()
df_a = pd.DataFrame()
df_h[names_home] = np.nan
df_a[names_away] = np.nan
for index, row in X_test.iterrows():
  home = (players_test[players_test['id'] == int(row['home_team'])][names_home])
  away = (players_test[players_test['id'] == int(row['away_team'])][names_away])
  df_h = df_h.append(home, ignore_index=True)
  df_a = df_a.append(away, ignore_index=True)
X_r_test = X_test[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef']]
X_r_test.reset_index(drop=True, inplace=True)
a = pd.concat([df_h, df_a], axis = 1)
a.reset_index(drop=True, inplace=True)
X_r_test = pd.concat([X_r_test, a], axis = 1)
teams_res_shots_test = np.zeros(X_r_test.shape[0])
teams_res_itog_test = np.zeros(X_r_test.shape[0])
for index, row in X_r_test.iterrows():
  hteam = int(row['home_team'])
  ateam = int(row['away_team'])
  teams_res_shots_test[index] = results_games_shots_test[hteam][ateam]
  teams_res_itog_test[index] = results_games_itog_test[hteam][ateam]
X_r_test['personal_res_shots'] = teams_res_shots_test
X_r_test['personal_res_itog'] = teams_res_itog_test
X_r_test['strong_personal_res_shots'] = (teams_res_shots_test > 8) * 1
X_r_test['weak_personal_shots'] = (teams_res_shots_test < -9) * 1
X_r_test['personal_res_shots_in_qube'] = teams_res_shots_test ** 3
X_r_test['personal_res_itog_in_qube'] = teams_res_itog_test ** 3

In [220]:
X_r_test['Division'] = X_r_test['Division'].apply(str)
X_r_test['home_team'] = X_r_test['home_team'].apply(str)
X_r_test['away_team'] = X_r_test['away_team'].apply(str)
X_r_test['Referee'] = X_r_test['Referee'].apply(str)
print(X_r_test.dtypes)

Division                         object
Time                            float64
home_team                        object
away_team                        object
Referee                          object
home_coef                       float64
draw_coef                       float64
away_coef                       float64
players_full_time_home_goals    float64
players_half_time_home_goals    float64
players_home_shots              float64
players_home_shots_on_target    float64
players_home_fouls              float64
players_home_corners            float64
players_home_yellow_cards       float64
players_home_red_cards          float64
players_home_coef               float64
players_full_time_away_goals    float64
players_half_time_away_goals    float64
players_away_shots              float64
players_away_shots_on_target    float64
players_away_fouls              float64
players_away_corners            float64
players_away_yellow_cards       float64
players_away_red_cards          float64


# Блок про рандомные модели

In [224]:
cat = cb.CatBoostClassifier(cat_features=[0, 2, 3, 4], l2_leaf_reg = 3, learning_rate = 0.9, depth=6, grow_policy='SymmetricTree')
cat = cat.fit(X_train, y_train, eval_set=(X_r_test, y_test), logging_level = 'Verbose')


0:	learn: 1.0171193	test: 1.0293001	best: 1.0293001 (0)	total: 74.5ms	remaining: 1m 14s
1:	learn: 1.0093397	test: 1.0289537	best: 1.0289537 (1)	total: 156ms	remaining: 1m 17s
2:	learn: 0.8754423	test: 1.1095069	best: 1.0289537 (1)	total: 223ms	remaining: 1m 14s
3:	learn: 0.8731583	test: 1.1103774	best: 1.0289537 (1)	total: 294ms	remaining: 1m 13s
4:	learn: 0.8710764	test: 1.1092323	best: 1.0289537 (1)	total: 367ms	remaining: 1m 13s
5:	learn: 0.8671693	test: 1.1128220	best: 1.0289537 (1)	total: 437ms	remaining: 1m 12s
6:	learn: 0.8639571	test: 1.1129772	best: 1.0289537 (1)	total: 518ms	remaining: 1m 13s
7:	learn: 0.8639237	test: 1.1129554	best: 1.0289537 (1)	total: 544ms	remaining: 1m 7s
8:	learn: 0.8344320	test: 1.1051091	best: 1.0289537 (1)	total: 609ms	remaining: 1m 7s
9:	learn: 0.8314437	test: 1.1070280	best: 1.0289537 (1)	total: 682ms	remaining: 1m 7s
10:	learn: 0.8274591	test: 1.1084009	best: 1.0289537 (1)	total: 778ms	remaining: 1m 9s
11:	learn: 0.8247471	test: 1.1100279	best: 1.

In [191]:
print(cat.score(X_train,y_train))

0.6070489482200647


# Блок про настоящий подсчёт

In [192]:
testPriznak.shape

(16491, 27)

In [193]:
print(cat.score(X_r_test,y_test))

0.4771556166265888


In [194]:
y_test

11962    1
30796    0
19317    0
7672     1
96      -1
        ..
27052    1
5211     1
89       0
3707     0
2495     0
Name: result, Length: 2911, dtype: int64

In [195]:
predict = cat.predict_proba(X_r_test)
real_result = y_test.to_numpy()
coefs = X_r_test[['away_coef', 'draw_coef', 'home_coef']].to_numpy()
yLose = (real_result == -1) * 1
errorLose = abs(predict[:, 0] - yLose)
yDraw = (real_result == 0) * 1
errorDraw = abs(predict[:, 1] - yDraw)
yWin = (real_result == 1) * 1
errorWin = abs(predict[:, 2] - yWin)
print(errorLose.mean())
print(errorDraw.mean())
print(errorWin.mean())

0.39854816067801563
0.39361476515806026
0.4599267729454823


In [196]:
predict

array([[0.32490566, 0.28835323, 0.38674111],
       [0.3057632 , 0.2885498 , 0.405687  ],
       [0.37101633, 0.27647718, 0.35250649],
       ...,
       [0.16579882, 0.29845075, 0.53575043],
       [0.26831545, 0.29723681, 0.43444774],
       [0.15652104, 0.21501936, 0.6284596 ]])

In [215]:
moneys = 0
for i, row in X_r_test.iterrows():
  hteam = int(float((row['home_team'])))
  ateam = int(float((row['away_team'])))
  if predict[i][0] > 0.44:
    moneys -= 1
    if real_result[i] == -1:
      moneys += coefs[i][0]
  elif predict[i][1] > 0.44:
    moneys -= 1
    if real_result[i] == 0:
      moneys += coefs[i][1]
  elif predict[i][2] > 0.44:
    moneys -= 1
    if real_result[i] == 1:
      moneys += coefs[i][2]
print(moneys)

15.045982131629621


# Блок про слияние

In [198]:
X_r = X[['Division', 'Time', 'home_team', 'away_team', 'Referee', 'home_coef', 'draw_coef', 'away_coef']]
X_r.reset_index(drop=True, inplace=True)
a = pd.concat([df_h, df_a], axis = 1)
a.reset_index(drop=True, inplace=True)
X_r = pd.concat([X_r, a], axis = 1)

In [199]:
teams_res_shots = np.zeros(X_r.shape[0])
teams_res_itog = np.zeros(X_r.shape[0])
for index, row in X_r.iterrows():
  hteam = int(row['home_team'])
  ateam = int(row['away_team'])
  teams_res_shots[index] = results_games_shots[hteam][ateam]
  teams_res_itog[index] = results_games_itog[hteam][ateam]

In [200]:
X_r['personal_res_shots'] = teams_res_shots
X_r['personal_res_itog'] = teams_res_itog
X_r['strong_personal_res_shots'] = (teams_res_shots > 8) * 1
X_r['weak_personal_shots'] = (teams_res_shots < -9) * 1
X_r['personal_res_shots_in_qube'] = teams_res_shots ** 3
X_r['personal_res_itog_in_qube'] = teams_res_itog ** 3

In [201]:
normalized_X=(X_r-X_r.mean())/X_r.std()
#tX = normalized_X.copy()
#tX['ans'] = y_train['result']
X_corr = X_r.copy()
X_corr['ans'] = y
cr = X_corr.corr()
cr

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots,personal_res_shots_in_qube,personal_res_itog_in_qube,ans
Division,1.000000,NaN,0.028651,0.031453,0.255949,-0.024717,-0.135642,-0.030124,-0.011330,-0.000457,-0.005533,-0.002363,0.003161,-0.023567,0.000157,0.001422,0.013378,-0.001387,0.001102,0.016567,0.012354,-0.000175,-0.010538,-0.010493,0.006685,-0.006445,0.001403,0.002176,-0.072924,-0.065878,-0.002024,-0.003447,-0.010284
Time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_team,0.028651,NaN,1.000000,-0.013134,-0.000700,-0.010251,-0.028000,-0.006728,0.021728,0.019403,-0.007496,0.000249,-0.019295,0.011087,-0.009154,0.016362,0.006739,0.004772,0.010127,-0.029484,-0.013582,-0.005133,-0.038141,0.006649,0.027341,0.007638,-0.029533,-0.026125,-0.058839,-0.011600,-0.053417,-0.048984,0.017884
away_team,0.031453,NaN,-0.013134,1.000000,0.004137,-0.029763,-0.025461,-0.001805,-0.059373,-0.058143,-0.036886,-0.037790,-0.014357,-0.023903,-0.019528,0.000133,0.038383,0.009232,0.006431,0.011401,0.009323,0.005445,0.024008,0.000849,-0.020809,-0.008721,0.034733,0.031877,-0.004811,-0.072775,0.052874,0.047158,-0.007192
Referee,0.255949,NaN,-0.000700,0.004137,1.000000,0.000468,-0.027921,-0.055996,0.018301,0.012461,-0.014996,-0.021285,-0.014890,-0.029208,-0.023001,-0.018002,-0.000869,-0.002908,0.003530,-0.011740,0.011169,-0.008524,0.007319,-0.039000,-0.025154,-0.001630,0.004379,0.003953,0.007301,0.010388,0.002916,0.002152,-0.015327
home_coef,-0.024717,NaN,-0.010251,-0.029763,0.000468,1.000000,0.089253,-0.453482,-0.026989,-0.028170,-0.018803,-0.013106,0.014866,-0.031452,0.028094,-0.016795,0.019486,-0.007758,-0.009505,0.005386,0.015848,-0.003274,-0.018483,-0.012881,-0.016981,-0.000837,-0.388393,-0.348175,-0.082178,0.277946,-0.282399,-0.256843,0.012672
draw_coef,-0.135642,NaN,-0.028000,-0.025461,-0.027921,0.089253,1.000000,0.793357,-0.030357,-0.029379,-0.020419,-0.026159,0.013623,-0.029367,0.041676,0.043287,0.009922,-0.006448,-0.009863,-0.001114,-0.007419,-0.007933,0.005279,-0.008852,-0.014333,0.019857,0.194664,0.181709,0.255460,0.082288,0.130799,0.130206,0.029061
away_coef,-0.030124,NaN,-0.006728,-0.001805,-0.055996,-0.453482,0.793357,1.000000,-0.012201,-0.009896,-0.011204,-0.014402,0.014517,-0.007363,0.027691,0.053742,-0.002037,0.004555,0.002931,-0.002163,-0.012867,0.004803,0.008602,0.006715,-0.004019,0.020328,0.385097,0.350073,0.260755,-0.083003,0.270341,0.251955,0.019125
players_full_time_home_goals,-0.011330,NaN,0.021728,-0.059373,0.018301,-0.026989,-0.030357,-0.012201,1.000000,0.891751,0.416366,0.546669,0.004151,0.415313,-0.038721,-0.120674,-0.616897,0.071629,0.066051,0.052554,0.031517,0.030452,-0.002318,0.030742,0.014058,0.065454,0.000997,0.006503,-0.015938,0.025252,-0.036417,-0.026817,0.000415
players_half_time_home_goals,-0.000457,NaN,0.019403,-0.058143,0.012461,-0.028170,-0.029379,-0.009896,0.891751,1.000000,0.330014,0.475717,-0.008997,0.363690,-0.074308,-0.118997,-0.537331,0.065145,0.063954,0.012276,0.026864,-0.007341,0.030628,-0.003037,0.005482,0.069527,-0.003948,0.005711,-0.024134,0.025668,-0.044745,-0.034419,-0.006607


In [202]:
print(X_r.dtypes)
X_r['Division'] = X_r['Division'].apply(str)
X_r['home_team'] = X_r['home_team'].apply(str)
X_r['away_team'] = X_r['away_team'].apply(str)
X_r['Referee'] = X_r['Referee'].apply(str)

Division                          int64
Time                            float64
home_team                       float64
away_team                       float64
Referee                           int64
home_coef                       float64
draw_coef                       float64
away_coef                       float64
players_full_time_home_goals    float64
players_half_time_home_goals    float64
players_home_shots              float64
players_home_shots_on_target    float64
players_home_fouls              float64
players_home_corners            float64
players_home_yellow_cards       float64
players_home_red_cards          float64
players_home_coef               float64
players_full_time_away_goals    float64
players_half_time_away_goals    float64
players_away_shots              float64
players_away_shots_on_target    float64
players_away_fouls              float64
players_away_corners            float64
players_away_yellow_cards       float64
players_away_red_cards          float64


# Блок про модели

In [203]:
y.shape

(16491,)

In [204]:
X_r.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)
data_tmp = pd.concat([X_r, y], axis = 1)
data_tmp['Division'] = data_tmp['Division'].apply(int)

In [205]:
data_tmp

,Division,Time,home_team,away_team,Referee,home_coef,draw_coef,away_coef,players_full_time_home_goals,players_half_time_home_goals,players_home_shots,players_home_shots_on_target,players_home_fouls,players_home_corners,players_home_yellow_cards,players_home_red_cards,players_home_coef,players_full_time_away_goals,players_half_time_away_goals,players_away_shots,players_away_shots_on_target,players_away_fouls,players_away_corners,players_away_yellow_cards,players_away_red_cards,players_away_coef,personal_res_shots,personal_res_itog,strong_personal_res_shots,weak_personal_shots,personal_res_shots_in_qube,personal_res_itog_in_qube,result
0,7,NaN,80.0,181.0,19,1.985403,3.534017,4.477083,1.140351,0.438596,11.614035,5.000000,10.122807,5.280702,1.192982,0.157895,2.676500,1.189189,0.432432,8.324324,3.567568,8.756757,5.405405,1.783784,0.054054,4.291101,0.0,0.0,0,0,0.0,0.0,1
1,8,NaN,419.0,167.0,-1,1.202023,7.803135,20.033724,1.317647,0.600000,12.095412,4.821160,12.810257,5.522804,1.767884,0.117368,2.783493,1.487179,0.717949,9.939089,4.025860,12.786451,4.438998,2.216123,0.102423,4.146168,1.0,1.0,0,0,1.0,1.0,1
2,6,NaN,271.0,407.0,-1,2.099905,3.573877,3.937322,1.278481,0.544304,12.088608,5.518987,9.025316,5.924051,1.354430,0.037975,2.981970,1.147368,0.400000,10.694737,4.621053,11.073684,5.389474,2.052632,0.136842,3.370373,0.0,0.0,0,0,0.0,0.0,1
3,0,NaN,176.0,129.0,-1,2.779840,3.246529,2.921876,1.529915,0.683761,13.222222,4.794872,10.846154,5.846154,1.196581,0.051282,2.133070,0.982301,0.389381,9.362832,3.575221,11.637168,5.053097,1.884956,0.115044,3.963592,1.0,1.0,0,0,1.0,1.0,1
4,6,NaN,461.0,16.0,-1,1.576569,4.042484,7.791889,1.376471,0.658824,12.257466,4.677563,12.225340,5.763959,1.724950,0.095848,2.875667,1.277108,0.578313,10.411533,4.012321,12.179490,4.655859,1.813062,0.107795,4.637262,0.0,0.0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16486,1,NaN,141.0,300.0,42,2.003596,3.706653,4.147444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,5.0,1,0,1331.0,125.0,1
16487,0,NaN,63.0,75.0,-1,3.001817,3.447248,2.585436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,0,0,-1.0,-1.0,1
16488,3,NaN,17.0,175.0,-1,3.847326,4.082043,1.979791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.0,-2.0,0,0,-216.0,-8.0,-1
16489,1,NaN,208.0,300.0,107,1.746202,3.903866,5.519605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0.0,0.0,1


In [206]:
yt = data_tmp['result'].copy()
del data_tmp['result']
Xt_r = data_tmp

In [207]:
Xt_r['Division'] = Xt_r['Division'].apply(str)
Xt_r['home_team'] = Xt_r['home_team'].apply(str)
Xt_r['away_team'] = Xt_r['away_team'].apply(str)
Xt_r.dtypes
Xt_r.reset_index(drop=True, inplace=True)
yt.reset_index(drop=True, inplace=True)

In [208]:
yt.isna().sum()

0

In [209]:
Xt_r.isna().sum()

Division                            0
Time                            16491
home_team                           0
away_team                           0
Referee                             0
home_coef                           0
draw_coef                           0
away_coef                           0
players_full_time_home_goals    13580
players_half_time_home_goals    13580
players_home_shots              13580
players_home_shots_on_target    13580
players_home_fouls              13580
players_home_corners            13580
players_home_yellow_cards       13580
players_home_red_cards          13580
players_home_coef               13580
players_full_time_away_goals    13580
players_half_time_away_goals    13580
players_away_shots              13580
players_away_shots_on_target    13580
players_away_fouls              13580
players_away_corners            13580
players_away_yellow_cards       13580
players_away_red_cards          13580
players_away_coef               13580
personal_res

In [210]:
cat = cb.CatBoostClassifier(cat_features=[0, 2, 3, 4])
grid = {'learning_rate': [0.03, 0.05, 0.6, 0.075, 0.875, 0.1, 0.11, 0.125, 0.5, 0.8, 1],
        'depth': [3, 4, 5, 6, 7, 8, 9, 10],
        'l2_leaf_reg': [0.1, 0.5, 1, 1.6, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.5, 4, 5, 7, 9, 11],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
        'auto_class_weights': ['None', 'Balanced', 'SqrtBalanced'],
        'od_pval': [0, 0.0000000001, 0.0000001, 0.00001, 0.0001, 0.001, 0.01],
        'posterior_sampling': [True, False]}
cat.randomized_search(grid, Xt_r, yt, n_iter = 3, cv = 4)
#cat = cat.fit(normalized_X,y_train['result'], verbose=False)

0:	learn: 1.0097010	test: 1.0189377	best: 1.0189377 (0)	total: 239ms	remaining: 3m 58s
1:	learn: 0.9891277	test: 1.0027347	best: 1.0027347 (1)	total: 497ms	remaining: 4m 7s
2:	learn: 0.9779448	test: 0.9953940	best: 0.9953940 (2)	total: 690ms	remaining: 3m 49s
3:	learn: 0.9713244	test: 0.9919920	best: 0.9919920 (3)	total: 905ms	remaining: 3m 45s
4:	learn: 0.9649414	test: 0.9917024	best: 0.9917024 (4)	total: 1.17s	remaining: 3m 52s
5:	learn: 0.9617090	test: 0.9915213	best: 0.9915213 (5)	total: 1.39s	remaining: 3m 50s
6:	learn: 0.9585442	test: 0.9909390	best: 0.9909390 (6)	total: 1.61s	remaining: 3m 47s
7:	learn: 0.9551236	test: 0.9905508	best: 0.9905508 (7)	total: 1.8s	remaining: 3m 43s
8:	learn: 0.9524415	test: 0.9909835	best: 0.9905508 (7)	total: 1.97s	remaining: 3m 36s
9:	learn: 0.9491152	test: 0.9898542	best: 0.9898542 (9)	total: 2.15s	remaining: 3m 32s
10:	learn: 0.9457304	test: 0.9902878	best: 0.9898542 (9)	total: 2.35s	remaining: 3m 31s
11:	learn: 0.9424327	test: 0.9905534	best: 0

KeyboardInterrupt: ignored

In [ ]:
Xt_r

In [ ]:
print(cat.score(Xt_r,yt))